In [2]:
# A smaller queue will lead to a shorter tail whereas a larger queue will create a longer tail
# (since more points are being tracked)

from collections import deque
from imutils.video import VideoStream
import numpy as np
import cv2
import imutils
import time
import argparse

In [3]:
buffer = 64
redlow = (150, 22, 6)
redup = (255,64,255)
pts = deque(maxlen = buffer)

# if video available
# vs = VideoStream(src=0).start()

# if no video path, capture from webcam
cap = cv2.VideoCapture(0)
if (cap.isOpened()== False): 
    print("Error opening video stream or file")


# time.sleep(2.0)

In [4]:
while cap.isOpened():
    ret , frame = cap.read()
    if ret == True:
#         cv2.imshow('Frame',frame) # display frame
        
        frame = imutils.resize(frame, width=600)
        blurr = cv2.GaussianBlur(frame, (11,11), 0)
        hsv = cv2.cvtColor(blurr, cv2.COLOR_BGR2HSV)

        mask = cv2.inRange(hsv, redlow, redup)
        mask = cv2.erode(mask, None, iterations = 2)
        mask = cv2.dilate(mask, None, iterations = 2)
        # find contours in the mask
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0]
#         cnts = imutils.grab_contours(cnts)
        # initialize center of the object
        center = None
        
        if len(cnts) > 0: # proceed only ifatleast 1 contour is found
            # largest contour in the mask
            c = max(cnts, key = cv2.contourArea)
            # minimum enclosing circle
            ((x,y), radius) = cv2.minEnclosingCircle(c)
            # centroid
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            
            if radius > 10:
                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                cv2.circle(frame, center, 5, (0,0,255), -1)
                
        pts.appendleft(center)
        
        for i in range(1, len(pts)):
            # if either of the tracked points are None, ignore them
            if pts[i-1] is None or pts[i] is None:
                continue
                
            # else compute thickness of line and draw connecting lines
            thickness = int(np.sqrt(buffer/ float(i+1)) * 1.5)
            cv2.line(frame, pts[i-1], pts[i], (0,0,255), thickness)
            
        cv2.imshow('Frame',frame) # display frame
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
            
    else:
        break
    
cap.release()
